# Lista 07 - Aprendizado de Máquina

## Aluno: Ruan Felipe de Almeida Silva - 397048

In [1]:
import pandas as pd
import numpy as np
from sklearn import metrics
from sklearn.neighbors import KNeighborsClassifier
from sklearn import model_selection
from sklearn.pipeline import Pipeline
from sklearn import preprocessing
from sklearn import pipeline
from sklearn import datasets

### Avalie o desempenho do algoritmo kNN para o dataset "Pima Indians Diabetes Database". Separe os dados em treino (80%) e teste (20%).


#### Importação dos dados

In [2]:
# dataframe = pd.read_csv('/content/drive/MyDrive/ufc - ec/Aprend_Maquina/datasets/diabetes.csv', delimiter=',')
dataframe = pd.read_csv('diabetes.csv', delimiter=',')
dataframe.head()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
0,6,148,72,35,0,33.6,0.627,50,1
1,1,85,66,29,0,26.6,0.351,31,0
2,8,183,64,0,0,23.3,0.672,32,1
3,1,89,66,23,94,28.1,0.167,21,0
4,0,137,40,35,168,43.1,2.288,33,1


#### Verificando a quantidade de linhas e colunas

In [3]:
dataframe.shape

(768, 9)

#### Verificando os tipos de dados do dataset

In [4]:
dataframe.dtypes

Pregnancies                   int64
Glucose                       int64
BloodPressure                 int64
SkinThickness                 int64
Insulin                       int64
BMI                         float64
DiabetesPedigreeFunction    float64
Age                           int64
Outcome                       int64
dtype: object

#### Verificando se existem valores faltantes no conjunto de dados

In [5]:
dataframe.isna().sum()

Pregnancies                 0
Glucose                     0
BloodPressure               0
SkinThickness               0
Insulin                     0
BMI                         0
DiabetesPedigreeFunction    0
Age                         0
Outcome                     0
dtype: int64

#### Analisando algumas informações sobre os dados para cada coluna

In [6]:
dataframe.describe()

,Pregnancies,Glucose,BloodPressure,SkinThickness,Insulin,BMI,DiabetesPedigreeFunction,Age,Outcome
count,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000,768.000000
mean,3.845052,120.894531,69.105469,20.536458,79.799479,31.992578,0.471876,33.240885,0.348958
std,3.369578,31.972618,19.355807,15.952218,115.244002,7.884160,0.331329,11.760232,0.476951
min,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.078000,21.000000,0.000000
25%,1.000000,99.000000,62.000000,0.000000,0.000000,27.300000,0.243750,24.000000,0.000000
50%,3.000000,117.000000,72.000000,23.000000,30.500000,32.000000,0.372500,29.000000,0.000000
75%,6.000000,140.250000,80.000000,32.000000,127.250000,36.600000,0.626250,41.000000,1.000000
max,17.000000,199.000000,122.000000,99.000000,846.000000,67.100000,2.420000,81.000000,1.000000


#### Separando as features do target e listando os possíveis valores para o target

In [7]:
X = dataframe.iloc[:,:-1]
y = dataframe.iloc[:,-1]
np.unique(y)

array([0, 1])

#### Verificando a quantidade de amostras para cada classe do target

In [8]:
y.value_counts()

0    500
1    268
Name: Outcome, dtype: int64

#### Dividindo os dados em 80% para o conjunto de treino e 20% para o conjunto de teste

In [9]:
X_values = X.values
y_values = y.values
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_values, y_values, test_size=0.2, random_state=142, stratify=y_values)
print("Conjunto x de treino:", len(X_train))
print("Conjunto x de teste:", len(X_test))
print("Conjunto y de treino:", len(y_train))
print("Conjunto y de teste:", len(y_test))

Conjunto x de treino: 614
Conjunto x de teste: 154
Conjunto y de treino: 614
Conjunto y de teste: 154


### 1. Use validação holdout para obter o melhor valor de k para o kNN com e sem escalonamento de features. Para isso, divida o conjunto de treino (80%) entre treino_holdout (70% do conjunto de treino) e validacao_holdout (30% do conjunto de treino). Use a métrica F1-Score para obtenção do melhor resultado. Armazene os resultados e faça um ranking deles (Grid Search manual). Mostre os resultados do melhor até o pior valor para o conjunto de validação (validacao_holdout). Agora treine um modelo com o melhor valor de k com todos os dados de treino (80%). Avalie o modelo final com os dados de teste (20%).

* Dicas: (1) use o Pipeline do scikit learn para realizar o escalonamento de features. (2) Não use nessa questão nem o cross_val_score, nem o GridSearchCV do scikit learn.

#### Dividindo o conjunto de treino em 70% para treino_holdout e 30% para validação_holdout

In [10]:
X_train_holdout, X_validacao_holdout, y_train_holdout, y_validacao_holdout = model_selection.train_test_split(X_train, y_train, test_size=0.3, random_state=142, stratify=y_train)
print("Conjunto x de treino holdout:", len(X_train_holdout))
print("Conjunto x de validação holdout:", len(X_validacao_holdout))
print("Conjunto y de treino holdout:", len(y_train_holdout))
print("Conjunto y de validação holdout", len(y_validacao_holdout))

Conjunto x de treino holdout: 429
Conjunto x de validação holdout: 185
Conjunto y de treino holdout: 429
Conjunto y de validação holdout 185


#### Treinando modelos de KNN com o conjunto de dados treino_holdout, para k variando de 1 a 20. Os modelos são testados com os dados de validação_holdout e as métricas, f1-score e acurácia são analisadas. 

In [11]:
array_k = np.arange(1, 21)

std = preprocessing.StandardScaler()
norm = preprocessing.MinMaxScaler()

scalers = ['no scaler', std, norm]
results = []

for k in array_k:
    for scaler in scalers:
        knn = KNeighborsClassifier(n_neighbors=k)
        if scaler == 'no scaler':
            pipe = Pipeline([('estimator', knn)])
            scl = 'No Scaler'
        else:
            pipe = Pipeline([('transformer', scaler), ('estimator', knn)])
            s = str(scaler)
            ind_parenteses = s.find('(')
            scl = s[:ind_parenteses]
            

        pipe.fit(X_train_holdout, y_train_holdout)
        y_pred = pipe.predict(X_validacao_holdout)
        accuracy = metrics.accuracy_score(y_validacao_holdout, y_pred)
        f1score = metrics.f1_score(y_validacao_holdout, y_pred, average='weighted', zero_division=0)
        
        print(f'Escalonamento: {scl}')
        print(f'kNN: k = {k}')
        print('Acurácia: {:.5f}'.format(accuracy))
        print('F1-Score: {:.5f}'.format(f1score))        
        print("\n---------------------------------------------------------------------------------\n")

        
        resultado = {'Value of k':k, 'Scaler':scl, 'Accuracy':accuracy, 'F1-Score':f1score}
        results.append(resultado)

Escalonamento: No Scaler
kNN: k = 1
Acurácia: 0.69189
F1-Score: 0.69349

---------------------------------------------------------------------------------

Escalonamento: StandardScaler
kNN: k = 1
Acurácia: 0.74054
F1-Score: 0.73325

---------------------------------------------------------------------------------

Escalonamento: MinMaxScaler
kNN: k = 1
Acurácia: 0.72432
F1-Score: 0.71884

---------------------------------------------------------------------------------

Escalonamento: No Scaler
kNN: k = 2
Acurácia: 0.72432
F1-Score: 0.68987

---------------------------------------------------------------------------------

Escalonamento: StandardScaler
kNN: k = 2
Acurácia: 0.71892
F1-Score: 0.67497

---------------------------------------------------------------------------------

Escalonamento: MinMaxScaler
kNN: k = 2
Acurácia: 0.70270
F1-Score: 0.66554

---------------------------------------------------------------------------------

Escalonamento: No Scaler
kNN: k = 3
Acurácia: 0.

#### Montando um dataframe com os resultados das métricas de cada modelo

In [12]:
df_results = pd.DataFrame(results)
df_results.sort_values(by='F1-Score', ascending=False)

,Value of k,Scaler,Accuracy,F1-Score
14,5,MinMaxScaler,0.800000,0.793794
38,13,MinMaxScaler,0.794595,0.787599
56,19,MinMaxScaler,0.783784,0.772016
32,11,MinMaxScaler,0.778378,0.771501
26,9,MinMaxScaler,0.778378,0.771501
30,11,No Scaler,0.772973,0.765240
16,6,StandardScaler,0.778378,0.761858
49,17,StandardScaler,0.772973,0.760617
25,9,StandardScaler,0.767568,0.760355
44,15,MinMaxScaler,0.767568,0.757397


#### Escolhendo os parâmetros do modelo que conseguiu o melhor f1-score

In [13]:
best_params = df_results.sort_values(by='F1-Score', ascending=False).iloc[0, :]
best_params

Value of k               5
Scaler        MinMaxScaler
Accuracy               0.8
F1-Score          0.793794
Name: 14, dtype: object

#### Criando o modelo com os melhores parâmetros. Treinando o modelo agora com todo o conjunto de treino e validando-o com o conjunto de teste. Analisando as métricas f1-score e acurácia pra esse novo modelo de kNN.

In [14]:
norm = preprocessing.MinMaxScaler()

value_k = best_params['Value of k']
scaler = best_params['Scaler']


knn_melhor = KNeighborsClassifier(n_neighbors=value_k)
pipe = Pipeline([('transformer', norm), ('estimator', knn_melhor)])
pipe.fit(X_train, y_train)


y_pred = pipe.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
f1score = metrics.f1_score(y_test, y_pred, average='weighted', zero_division=0)

print(f'Escalonamento: {scaler}')
print(f'kNN: k = {value_k}')
print('Acurácia: {:.5f}'.format(accuracy))
print('F1-Score: {:.5f}'.format(f1score))        


Escalonamento: MinMaxScaler
kNN: k = 5
Acurácia: 0.69481
F1-Score: 0.69661


#### Montando um dataframe com apenas os modelos que não utilizaram escalonamento. Ordenando em ordem decrescente da métrica F1-Score

In [15]:
df_noScaler = df_results[df_results['Scaler'] == 'No Scaler']
df_noScaler.sort_values(by='F1-Score', ascending=False).head(10)

,Value of k,Scaler,Accuracy,F1-Score
30,11,No Scaler,0.772973,0.765240
21,8,No Scaler,0.767568,0.757397
24,9,No Scaler,0.762162,0.756803
18,7,No Scaler,0.756757,0.754275
36,13,No Scaler,0.756757,0.747711
42,15,No Scaler,0.756757,0.747711
27,10,No Scaler,0.756757,0.744410
48,17,No Scaler,0.745946,0.734829
15,6,No Scaler,0.740541,0.731703
39,14,No Scaler,0.740541,0.724426


#### Escolhendo os parâmetros do modelo sem escalonamento que conseguiu o melhor f1-score

In [16]:
best_params = df_noScaler.sort_values(by='F1-Score', ascending=False).iloc[0, :]
best_params

Value of k           11
Scaler        No Scaler
Accuracy       0.772973
F1-Score        0.76524
Name: 30, dtype: object

#### Criando o modelo com os melhores parâmetros. Treinando o modelo agora com todo o conjunto de treino e validando-o com o conjunto de teste. Analisando as métricas f1-score e acurácia pra esse novo modelo de kNN.

In [17]:

value_k = best_params['Value of k']
scaler = best_params['Scaler']


knn_melhor = KNeighborsClassifier(n_neighbors=value_k)
knn_melhor.fit(X_train, y_train)


y_pred = knn_melhor.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
f1score = metrics.f1_score(y_test, y_pred, average='weighted', zero_division=0)

print(f'Escalonamento: {scaler}')
print(f'kNN: k = {value_k}')
print('Acurácia: {:.5f}'.format(accuracy))
print('F1-Score: {:.5f}'.format(f1score))    

Escalonamento: No Scaler
kNN: k = 11
Acurácia: 0.71429
F1-Score: 0.70451


### 2. Use validação cruzada com k-Fold=5 para obter o melhor valor de k para o kNN com e sem escalonamento de features. Use a métrica F1-Score para obtenção do melhor resultado. Armazene os resultados e faça um ranking deles (Grid Search manual). Mostre os resultados do melhor até o pior valor para o conjunto de validação. Agora treine um modelo com o melhor valor de k com todos os dados de treino (80%). Avalie o modelo final com os dados de teste (20%).

* Dicas: (1) use a função cross_val_score do scikit learn com cv=5. (2) use o Pipeline do scikit learn para realizar o escalonamento de features. (3) Não use nessa questão o GridSearchCV do scikit learn.

#### Treinando modelos de KNN, para k variando de 1 a 20, utilizando validação cruzada com o cross_val_score com 5 divisões diferentes para cada modelo e analisando a métrica f1-score

In [18]:
array_k = np.arange(1, 21)

std = preprocessing.StandardScaler()
norm = preprocessing.MinMaxScaler()

scalers = ['no scaler', std, norm]
results = []

for k in array_k:
    for scaler in scalers:
        knn = KNeighborsClassifier(n_neighbors=k)
        if scaler == 'no scaler':
            pipe = Pipeline([('estimator', knn)])
            scl = 'No Scaler'
        else:
            pipe = Pipeline([('transformer', scaler), ('estimator', knn)])
            s = str(scaler)
            ind_parenteses = s.find('(')
            scl = s[:ind_parenteses]
            

        f1score = model_selection.cross_val_score(pipe, X_train, y_train, cv=5, scoring='f1_weighted')
        
        print(f'Escalonamento: {scl}')
        print(f'kNN: k = {k}')
        print(f'F1-Score: {f1score}')        
        print("\n---------------------------------------------------------------------------------\n")

        
        resultado = {'Value of k':k, 'Scaler':scl, 'F1-Score':f1score.mean()}
        results.append(resultado)

Escalonamento: No Scaler
kNN: k = 1
F1-Score: [0.66493902 0.6939615  0.72357724 0.70163091 0.60856421]

---------------------------------------------------------------------------------

Escalonamento: StandardScaler
kNN: k = 1
F1-Score: [0.69684959 0.73666468 0.71292322 0.7375914  0.73742167]

---------------------------------------------------------------------------------

Escalonamento: MinMaxScaler
kNN: k = 1
F1-Score: [0.70513956 0.72749172 0.75469953 0.74722495 0.69049584]

---------------------------------------------------------------------------------

Escalonamento: No Scaler
kNN: k = 2
F1-Score: [0.70790481 0.68598813 0.69405851 0.69705182 0.64048092]

---------------------------------------------------------------------------------

Escalonamento: StandardScaler
kNN: k = 2
F1-Score: [0.66660744 0.67636369 0.7148648  0.69069014 0.70011774]

---------------------------------------------------------------------------------

Escalonamento: MinMaxScaler
kNN: k = 2
F1-Score: [0.

#### Montando um dataframe com os resultados da métrica f1-score de cada modelo e ordenando os modelos de acordo com essa métrica.

In [19]:
df_results = pd.DataFrame(results)
df_results.sort_values(by='F1-Score', ascending=False)

,Value of k,Scaler,F1-Score
19,7,StandardScaler,0.782250
38,13,MinMaxScaler,0.765790
25,9,StandardScaler,0.760645
20,7,MinMaxScaler,0.755049
32,11,MinMaxScaler,0.753592
31,11,StandardScaler,0.752885
44,15,MinMaxScaler,0.750870
22,8,StandardScaler,0.749179
36,13,No Scaler,0.748976
14,5,MinMaxScaler,0.748506


#### Escolhendo os parâmetros do modelo que conseguiu o melhor f1-score

In [20]:
best_params = df_results.sort_values(by='F1-Score', ascending=False).iloc[0, :]
best_params

Value of k                 7
Scaler        StandardScaler
F1-Score             0.78225
Name: 19, dtype: object

#### Criando o modelo com os melhores parâmetros. Treinando o modelo agora com todo o conjunto de treino e validando-o com o conjunto de teste. Analisando as métricas f1-score e acurácia pra esse novo modelo de kNN.

In [21]:
std = preprocessing.StandardScaler()

value_k = best_params['Value of k']
scaler = best_params['Scaler']


knn_melhor = KNeighborsClassifier(n_neighbors=value_k)
pipe = Pipeline([('transformer', std), ('estimator', knn_melhor)])
pipe.fit(X_train, y_train)


y_pred = pipe.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
f1score = metrics.f1_score(y_test, y_pred, average='weighted', zero_division=0)

print(f'Escalonamento: {scaler}')
print(f'kNN: k = {value_k}')
print('Acurácia: {:.5f}'.format(accuracy))
print('F1-Score: {:.5f}'.format(f1score)) 

Escalonamento: StandardScaler
kNN: k = 7
Acurácia: 0.71429
F1-Score: 0.71159


#### Montando um dataframe com apenas os modelos que não utilizaram escalonamento. Ordenando em ordem decrescente da métrica F1-Score

In [22]:
df_noScaler = df_results[df_results['Scaler'] == 'No Scaler']
df_noScaler.sort_values(by='F1-Score', ascending=False).head(10)

,Value of k,Scaler,F1-Score
36,13,No Scaler,0.748976
24,9,No Scaler,0.744879
18,7,No Scaler,0.743514
33,12,No Scaler,0.740375
27,10,No Scaler,0.737264
39,14,No Scaler,0.737213
21,8,No Scaler,0.734474
42,15,No Scaler,0.730599
30,11,No Scaler,0.729861
48,17,No Scaler,0.729360


#### Escolhendo os parâmetros do modelo sem escalonamento que conseguiu o melhor f1-score

In [23]:
best_params = df_noScaler.sort_values(by='F1-Score', ascending=False).iloc[0, :]
best_params

Value of k           13
Scaler        No Scaler
F1-Score       0.748976
Name: 36, dtype: object

#### Criando o modelo com os melhores parâmetros. Treinando o modelo agora com todo o conjunto de treino e validando-o com o conjunto de teste. Analisando as métricas f1-score e acurácia pra esse novo modelo de kNN.

In [24]:

value_k = best_params['Value of k']
scaler = best_params['Scaler']


knn_melhor = KNeighborsClassifier(n_neighbors=value_k)
knn_melhor.fit(X_train, y_train)


y_pred = knn_melhor.predict(X_test)
accuracy = metrics.accuracy_score(y_test, y_pred)
f1score = metrics.f1_score(y_test, y_pred, average='weighted', zero_division=0)

print(f'Escalonamento: {scaler}')
print(f'kNN: k = {value_k}')
print('Acurácia: {:.5f}'.format(accuracy))
print('F1-Score: {:.5f}'.format(f1score)) 

Escalonamento: No Scaler
kNN: k = 13
Acurácia: 0.70779
F1-Score: 0.70080


#### 3. Responda a questão 1 só que agora usando o GridSearchCV do scikit learn. Dica: Nesta questão você não deve usar o cross_val_score do scikit learn.

#### Dividindo os dados em 80% para o conjunto de treino e 20% para o conjunto de teste

In [25]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X_values, y_values, test_size=0.2, random_state=142, stratify=y_values)
print("Conjunto x de treino:", len(X_train))
print("Conjunto x de teste:", len(X_test))
print("Conjunto y de treino:", len(y_train))
print("Conjunto y de teste:", len(y_test))

Conjunto x de treino: 614
Conjunto x de teste: 154
Conjunto y de treino: 614
Conjunto y de teste: 154


Verificando os parâmetros existentes no classificador KNeighborsClassifier

In [26]:
model = KNeighborsClassifier()
model.get_params().keys()

dict_keys(['algorithm', 'leaf_size', 'metric', 'metric_params', 'n_jobs', 'n_neighbors', 'p', 'weights'])

#### Utilizando o GridSearchCV para encontrar os melhores modelos para cada escalonamento (No Scaler, Standard Scaler, MinMax Scaler), de acordo com a métrica f1-score, para os seguintes parâmetros: n_neighbors, weights, metric.

In [27]:
std = preprocessing.StandardScaler()
norm = preprocessing.MinMaxScaler()
scalers = ['no scaler', std, norm]
grid_results = []

for scaler in scalers:
    model = KNeighborsClassifier()
    if scaler == 'no scaler':
        pipe = pipeline.make_pipeline(model)
        scl = 'No Scaler'
    else:
        pipe = pipeline.make_pipeline(scaler, model)
        s = str(scaler)
        ind_parenteses = s.find('(')
        scl = s[:ind_parenteses]
            
    param_k = np.arange(1, 21)
    param_grid = {'kneighborsclassifier__n_neighbors': param_k, 
                  'kneighborsclassifier__weights':['uniform', 'distance'],
                  'kneighborsclassifier__metric':['manhattan', 'minkowski']}

    gridSearch = model_selection.GridSearchCV(estimator=pipe, param_grid=param_grid, return_train_score=False, scoring='f1_weighted', cv=5)
    gs = gridSearch.fit(X=X_train, y=y_train)
    print(f'Escalonamento: {scl}')
    print(f'Melhor F1-Score: {gs.best_score_}')        
    print(f'Melhores parâmetros: {gs.best_params_}')
    print("\n---------------------------------------------------------------------------------\n")
    
    grid_results.append(gs)



Escalonamento: No Scaler
Melhor F1-Score: 0.7572505826802554
Melhores parâmetros: {'kneighborsclassifier__metric': 'manhattan', 'kneighborsclassifier__n_neighbors': 12, 'kneighborsclassifier__weights': 'distance'}

---------------------------------------------------------------------------------

Escalonamento: StandardScaler
Melhor F1-Score: 0.7825835625547974
Melhores parâmetros: {'kneighborsclassifier__metric': 'minkowski', 'kneighborsclassifier__n_neighbors': 7, 'kneighborsclassifier__weights': 'distance'}

---------------------------------------------------------------------------------

Escalonamento: MinMaxScaler
Melhor F1-Score: 0.7657899989862841
Melhores parâmetros: {'kneighborsclassifier__metric': 'minkowski', 'kneighborsclassifier__n_neighbors': 13, 'kneighborsclassifier__weights': 'uniform'}

---------------------------------------------------------------------------------



#### Analisando as chaves retornadas no dicionário com os resultados de um dos GridSearchs.

In [28]:
keys = grid_results[0].cv_results_.keys()
keys

dict_keys(['mean_fit_time', 'std_fit_time', 'mean_score_time', 'std_score_time', 'param_kneighborsclassifier__metric', 'param_kneighborsclassifier__n_neighbors', 'param_kneighborsclassifier__weights', 'params', 'split0_test_score', 'split1_test_score', 'split2_test_score', 'split3_test_score', 'split4_test_score', 'mean_test_score', 'std_test_score', 'rank_test_score'])

#### Gerando um dataframe para o resultado do primeiro GridSearch, referente aos modelos que não utilizaram escalonamento. Esse dataframe é ordenado de acordo com rank_test_score, que foi organizado de acordo com a métrica escolhida, que nesse caso foi o f1-score.

In [29]:
df_results_noScaler = pd.DataFrame(grid_results[0].cv_results_)
df_results_noScaler_ordered = df_results_noScaler.sort_values(by='rank_test_score')
df_results_noScaler_ordered.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kneighborsclassifier__metric,param_kneighborsclassifier__n_neighbors,param_kneighborsclassifier__weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
23,0.001719,0.000468,0.004017,0.000859,manhattan,12,distance,"{'kneighborsclassifier__metric': 'manhattan', ...",0.749807,0.706394,0.792003,0.795864,0.742185,0.757251,0.033369,1
20,0.001415,0.000283,0.009500,0.001552,manhattan,11,uniform,"{'kneighborsclassifier__metric': 'manhattan', ...",0.758391,0.710807,0.749397,0.812875,0.747011,0.755696,0.032884,2
24,0.001799,0.000229,0.011494,0.002424,manhattan,13,uniform,"{'kneighborsclassifier__metric': 'manhattan', ...",0.739837,0.700802,0.756727,0.820481,0.756472,0.754864,0.038627,3
27,0.001682,0.000311,0.004468,0.000790,manhattan,14,distance,"{'kneighborsclassifier__metric': 'manhattan', ...",0.724969,0.710807,0.784511,0.795864,0.752031,0.753637,0.032849,4
39,0.001301,0.000079,0.003967,0.000696,manhattan,20,distance,"{'kneighborsclassifier__metric': 'manhattan', ...",0.716188,0.744830,0.780928,0.798953,0.717829,0.751745,0.033295,5


#### Gerando um dataframe para o resultado do segundo GridSearch, referente aos modelos que utilizaram o escalonamento Standard Scaler. Esse dataframe é ordenado de acordo com rank_test_score, que foi organizado de acordo com a métrica escolhida, que nesse caso foi o f1-score.

In [30]:
df_results_std = pd.DataFrame(grid_results[1].cv_results_)
df_results_std_ordered = df_results_std.sort_values(by='rank_test_score')
df_results_std_ordered.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kneighborsclassifier__metric,param_kneighborsclassifier__n_neighbors,param_kneighborsclassifier__weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
53,0.002581,0.000375,0.005003,0.001075,minkowski,7,distance,"{'kneighborsclassifier__metric': 'minkowski', ...",0.764841,0.766104,0.769582,0.835831,0.776560,0.782584,0.026933,1
52,0.002494,0.000567,0.010214,0.000909,minkowski,7,uniform,"{'kneighborsclassifier__metric': 'minkowski', ...",0.779878,0.749397,0.769582,0.835831,0.776560,0.782250,0.028805,2
55,0.002431,0.000438,0.004602,0.000528,minkowski,8,distance,"{'kneighborsclassifier__metric': 'minkowski', ...",0.757325,0.766104,0.767935,0.793758,0.776560,0.772336,0.012331,3
31,0.002726,0.000388,0.006434,0.000931,manhattan,16,distance,"{'kneighborsclassifier__metric': 'manhattan', ...",0.736665,0.769190,0.784511,0.771329,0.784021,0.769143,0.017419,4
59,0.002670,0.000558,0.004351,0.000141,minkowski,10,distance,"{'kneighborsclassifier__metric': 'minkowski', ...",0.778541,0.764077,0.749397,0.766708,0.781590,0.768062,0.011480,5


#### Gerando um dataframe para o resultado do terceiro GridSearch, referente aos modelos que utilizaram o escalonamento MinMax Scaler. Esse dataframe é ordenado de acordo com rank_test_score, que foi organizado de acordo com a métrica escolhida, que nesse caso foi o f1-score.

In [31]:
df_results_norm = pd.DataFrame(grid_results[2].cv_results_)
df_results_norm_ordered = df_results_norm.sort_values(by='rank_test_score')
df_results_norm_ordered.head()

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kneighborsclassifier__metric,param_kneighborsclassifier__n_neighbors,param_kneighborsclassifier__weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
64,0.001823,0.000156,0.010989,0.001280,minkowski,13,uniform,"{'kneighborsclassifier__metric': 'minkowski', ...",0.763573,0.776571,0.767935,0.781470,0.739401,0.765790,0.014615,1
23,0.002070,0.000309,0.005982,0.001085,manhattan,12,distance,"{'kneighborsclassifier__metric': 'manhattan', ...",0.754700,0.771451,0.780928,0.763988,0.743554,0.762924,0.012968,2
65,0.002145,0.000300,0.005503,0.000762,minkowski,13,distance,"{'kneighborsclassifier__metric': 'minkowski', ...",0.763573,0.759384,0.758723,0.781470,0.746645,0.761959,0.011269,3
35,0.002249,0.000331,0.006124,0.001029,manhattan,18,distance,"{'kneighborsclassifier__metric': 'manhattan', ...",0.699276,0.764077,0.792003,0.775706,0.761240,0.758460,0.031510,4
21,0.001954,0.000342,0.005862,0.000708,manhattan,11,distance,"{'kneighborsclassifier__metric': 'manhattan', ...",0.760523,0.764077,0.764077,0.752093,0.746645,0.757483,0.006967,5


#### Montando um dataframe apenas com o melhor resultado de cada um dos GridSearchs.

In [32]:
best_results = []
scalers = ['No Scaler', 'StandardScaler', 'MinMaxScaler']
for i in range(0, len(grid_results)):
    results = pd.DataFrame(grid_results[i].cv_results_)
    results_ordered = results.sort_values(by='rank_test_score')
    results_ordered['scaler'] = scalers[i]
    best_results.append(results_ordered.iloc[0, :])

df_results = pd.DataFrame(best_results)
df_results

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_kneighborsclassifier__metric,param_kneighborsclassifier__n_neighbors,param_kneighborsclassifier__weights,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score,scaler
23,0.001719,0.000468,0.004017,0.000859,manhattan,12,distance,"{'kneighborsclassifier__metric': 'manhattan', ...",0.749807,0.706394,0.792003,0.795864,0.742185,0.757251,0.033369,1,No Scaler
53,0.002581,0.000375,0.005003,0.001075,minkowski,7,distance,"{'kneighborsclassifier__metric': 'minkowski', ...",0.764841,0.766104,0.769582,0.835831,0.776560,0.782584,0.026933,1,StandardScaler
64,0.001823,0.000156,0.010989,0.001280,minkowski,13,uniform,"{'kneighborsclassifier__metric': 'minkowski', ...",0.763573,0.776571,0.767935,0.781470,0.739401,0.765790,0.014615,1,MinMaxScaler


#### Treinando os modelos com os melhores parâmetros encontrados pelo GridSearch para cada tipo de escalonamento. O modelo é avaliado no conjunto de teste e as métricas f1-score e acurácia são observadas.

In [33]:
std = preprocessing.StandardScaler()
norm = preprocessing.MinMaxScaler()
scalers = ['no scaler', std, norm]
for i in range(0, len(df_results)):
    row = df_results.iloc[i, :]
    n_neighbors = row['param_kneighborsclassifier__n_neighbors']
    metric = row['param_kneighborsclassifier__metric']
    weights = row['param_kneighborsclassifier__weights']
    scaler = row['scaler'] 
    
    model = KNeighborsClassifier(n_neighbors=n_neighbors, metric=metric, weights=weights)
    
    if i == 0:
        pipe = Pipeline([('estimator', model)])
    else:
        pipe = Pipeline([('transformer', scalers[i]), ('estimator', knn)])
    

    pipe.fit(X_train, y_train)
    y_pred = pipe.predict(X_test)
    accuracy = metrics.accuracy_score(y_test, y_pred)
    f1score = metrics.f1_score(y_test, y_pred, average='weighted', zero_division=0)

    print(f'Escalonamento: {scaler}')
    print(f'kNN: k = {n_neighbors}')
    print('Acurácia: {:.5f}'.format(accuracy))
    print('F1-Score: {:.5f}'.format(f1score))
    print("\n---------------------------------------------------------------------------------\n")

Escalonamento: No Scaler
kNN: k = 12
Acurácia: 0.69481
F1-Score: 0.69112

---------------------------------------------------------------------------------

Escalonamento: StandardScaler
kNN: k = 7
Acurácia: 0.74026
F1-Score: 0.72941

---------------------------------------------------------------------------------

Escalonamento: MinMaxScaler
kNN: k = 13
Acurácia: 0.72078
F1-Score: 0.71020

---------------------------------------------------------------------------------

